In [26]:
# pvcobra is a paid software thus not considered
import vad
import wave
import torchaudio
from vad.energy_vad import EnergyVAD

In [25]:
# load audio file in "audio" variable

vad1 = EnergyVAD(
    sample_rate = 16000,
    frame_length = 25,
    frame_shift = 20,
    energy_threshold = 0.05,
)

In [23]:
audio = wave.open('example.wav','r')
audio.getparams()

_wave_params(nchannels=2, sampwidth=3, framerate=44100, nframes=1408998, comptype='NONE', compname='not compressed')

In [27]:

waveform, sample_rate = torchaudio.load('example.wav')

In [29]:
waveform.numpy()

array([[ 0.0000000e+00, -1.1920929e-07, -1.1920929e-07, ...,
         0.0000000e+00, -1.1920929e-07,  0.0000000e+00],
       [ 0.0000000e+00, -1.1920929e-07, -1.1920929e-07, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [30]:

# Compute VAD
vad2 = EnergyVAD()
vad_output = vad2(waveform.numpy())

# print vad output 10 per line
print(len(vad_output))

4402


In [21]:
voice_activity = vad1(
    audio
)

AttributeError: 'Wave_read' object has no attribute 'ndim'

In [ ]:

speech_signal = vad1.apply_vad(audio)